In [1]:
!ls

sample_data


In [2]:
!pwd

/content


In [3]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   44G  183G  20% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.9G     0  5.9G   0% /dev/shm
/dev/root       2.0G  1.2G  821M  59% /sbin/docker-init
tmpfs           6.4G   28K  6.4G   1% /var/colab
/dev/sda1       233G   47G  186G  21% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!wget https://cvlab.s3.yandex.net/mlcup2021/images.json

--2021-10-04 08:10:40--  https://cvlab.s3.yandex.net/mlcup2021/images.json
Resolving cvlab.s3.yandex.net (cvlab.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to cvlab.s3.yandex.net (cvlab.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 605923636 (578M) [application/json]
Saving to: ‘images.json’

images.json         100%[===================>] 577.85M  22.9MB/s    in 27s     

2021-10-04 08:11:08 (21.8 MB/s) - ‘images.json’ saved [605923636/605923636]



In [7]:
BASE_PATH = '/content/drive/MyDrive/ai/ycup21/'

In [8]:
!ls {BASE_PATH}

config	 data	     main.py   notebooks   README.md  train.py
contest  dataset.py  model.py  predict.py  system.py


In [9]:
BASE_PATH_DATA = f'/content/drive/MyDrive/ai/ycup21/data'

In [6]:
# !wget https://cvlab.s3.yandex.net/mlcup2021/metadata.json

In [10]:
# !cp ./metadata.json {BASE_PATH_DATA}

In [11]:
import json
import os

import requests
from io import BytesIO
from PIL import Image

In [12]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [13]:
import warnings

warnings.filterwarnings("ignore")

In [14]:
# data_path = '../data'
data_path = './'

image_save_path = './images'
image_size = 512

In [15]:
!mkdir images

In [16]:
with open(os.path.join(data_path, 'images.json')) as json_file:
    json_strings = json_file.readlines()

In [17]:
len(json_strings)

5462418

In [18]:
for json_string in json_strings[:10]:
    image_data = json.loads(json_string)
    print(image_data['image'], image_data['url'])

1 http://static.pleer.ru/i/gp/826/344/norm.jpg
2 https://i0.wp.com/allwomens.ru/uploads/posts/2018-01/1515670311_mariya-gorban-10.jpg
3 https://99px.ru/sstorage/56/2015/10/image_561810151141106183724.png
4 https://grandmirage.ru/5158-large_default/stol-pismennyj-fierta-43-021.jpg
5 https://wallpaper-mania.com/wp-content/uploads/2018/09/High_resolution_wallpaper_background_ID_77700295631.jpg
6 https://drasler.ru/wp-content/uploads/2019/08/%D0%94%D0%BE%D0%B1%D1%80%D0%BE%D0%B5-%D1%83%D1%82%D1%80%D0%BE-%D0%BA%D0%B0%D1%80%D1%82%D0%B8%D0%BD%D0%BA%D0%B8-%D0%B4%D0%BB%D1%8F-%D0%BC%D1%83%D0%B6%D1%87%D0%B8%D0%BD%D1%8B-%D1%81-%D0%BD%D0%B0%D0%B4%D0%BF%D0%B8%D1%81%D1%8F%D0%BC%D0%B8-%D0%BA%D0%BE%D1%84%D0%B5-30.jpg
7 https://static.wikia.nocookie.net/fantasticbeings/images/d/d2/%D0%AE%D1%81%D1%82%D1%80%D0%B8%D1%86%D0%B0.jpg/revision/latest?cb=20200408081506&path-prefix=ru
8 https://img-fotki.yandex.ru/get/6512/20839000.3e/0_85e9c_8d598366_orig
9 https://avatars.mds.yandex.net/get-zen_doc/2816669/pub_5

In [19]:
json_strings[:10]

['{"image": 1, "url": "http://static.pleer.ru/i/gp/826/344/norm.jpg"}\n',
 '{"image": 2, "url": "https://i0.wp.com/allwomens.ru/uploads/posts/2018-01/1515670311_mariya-gorban-10.jpg"}\n',
 '{"image": 3, "url": "https://99px.ru/sstorage/56/2015/10/image_561810151141106183724.png"}\n',
 '{"image": 4, "url": "https://grandmirage.ru/5158-large_default/stol-pismennyj-fierta-43-021.jpg"}\n',
 '{"image": 5, "url": "https://wallpaper-mania.com/wp-content/uploads/2018/09/High_resolution_wallpaper_background_ID_77700295631.jpg"}\n',
 '{"image": 6, "url": "https://drasler.ru/wp-content/uploads/2019/08/%D0%94%D0%BE%D0%B1%D1%80%D0%BE%D0%B5-%D1%83%D1%82%D1%80%D0%BE-%D0%BA%D0%B0%D1%80%D1%82%D0%B8%D0%BD%D0%BA%D0%B8-%D0%B4%D0%BB%D1%8F-%D0%BC%D1%83%D0%B6%D1%87%D0%B8%D0%BD%D1%8B-%D1%81-%D0%BD%D0%B0%D0%B4%D0%BF%D0%B8%D1%81%D1%8F%D0%BC%D0%B8-%D0%BA%D0%BE%D1%84%D0%B5-30.jpg"}\n',
 '{"image": 7, "url": "https://static.wikia.nocookie.net/fantasticbeings/images/d/d2/%D0%AE%D1%81%D1%82%D1%80%D0%B8%D1%86%D0%B0.j

In [ ]:
index_to_remove = [12, 40, 57, 78, 88, 96, 97]

In [ ]:
json_strings_reduced = [json_string for index, json_string in enumerate(json_strings[:100]) if index not in index_to_remove]

In [ ]:
len(json_strings_reduced)

93

In [20]:
index_to_remove = []
id_to_remove = []

In [21]:
start_from = 0
num_images_to_load = 100000

In [ ]:
# TODO: do not wait more than 10sec

In [24]:
image_formats = ("image/png", "image/jpeg", "image/jpg")

def is_url_image(r):
   if "content-type" in r.headers and r.headers["content-type"] in image_formats:
      return True
   return False

In [26]:
t0 = time.time()

for index, json_string in enumerate(json_strings[start_from:num_images_to_load]):
    
#     if index in index_to_remove:
#         continue
    
    t1 = time.time()
    
    image_json = json.loads(json_string)
    
#     print(index, image_json['image'], image_json['url'])
      
    image_id = image_json['image']
    image_name = f'image_{image_id}.jpg'
      
    try:
        r = requests.get(image_json['url'], verify=False, timeout=10)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        print('Image couldn\'t be retreived: ', index, image_name, e, format_time(time.time() - t1))
        index_to_remove.append(index)
        id_to_remove.append(image_id)
        continue

    if r.status_code != 200:
        print('Image couldn\'t be retreived: ', index, image_name, r.status_code, format_time(time.time() - t1))
        index_to_remove.append(index)
        id_to_remove.append(image_id)
        continue

    if not is_url_image(r):
        print('Not an image: ', index, image_name, r.status_code, format_time(time.time() - t1))
        index_to_remove.append(index)
        id_to_remove.append(image_id)
        continue
       

    # print('Image sucessfully downloaded: ', index, image_name, format_time(time.time() - t1))
    # with open(os.path.join(image_save_path, image_name), 'wb') as handler:
    #     handler.write(r.content)

    image = Image.open(BytesIO(r.content))
    image = image.convert('RGB')
    # image = image.resize((image_size, image_size))
    image.thumbnail((image_size, image_size))
    image.save(os.path.join(image_save_path, image_name))
           
print('loading finished for: {}'.format(format_time(time.time() - t0)))

Image couldn't be retreived:  12 image_13.jpg 402 0:00:01
Not an image:  57 image_63.jpg 200 0:00:00
Image couldn't be retreived:  88 image_97.jpg 404 0:00:01
Image couldn't be retreived:  96 image_105.jpg 400 0:00:01
Not an image:  109 image_120.jpg 200 0:00:01
Image couldn't be retreived:  114 image_127.jpg 403 0:00:00
Image couldn't be retreived:  135 image_149.jpg 402 0:00:01
Not an image:  136 image_150.jpg 200 0:00:01
Image couldn't be retreived:  155 image_170.jpg 520 0:00:00
Not an image:  168 image_183.jpg 200 0:00:01
Image couldn't be retreived:  176 image_191.jpg 404 0:00:01
Image couldn't be retreived:  182 image_197.jpg HTTPSConnectionPool(host='static.mvd.ru', port=443): Read timed out. (read timeout=10) 0:00:14
Not an image:  199 image_217.jpg 200 0:00:00
Image couldn't be retreived:  208 image_230.jpg 403 0:00:00
Not an image:  223 image_247.jpg 200 0:00:01
Image couldn't be retreived:  225 image_249.jpg 402 0:00:01
Image couldn't be retreived:  243 image_267.jpg 402 0:

UnidentifiedImageError: ignored

In [27]:
# index_to_remove, id_to_remove
id_to_remove[-10:], index_to_remove[-10:]

([28841, 28850, 28852, 28864, 28908, 28919, 28954, 28959, 28960, 28965],
 [26499, 26507, 26509, 26519, 26561, 26572, 26605, 26609, 26610, 26615])

In [28]:
import numpy as np

In [29]:
np.save('index_to_remove.npy', index_to_remove)
np.save('id_to_remove.npy', id_to_remove)

In [36]:
len(index_to_remove), len(id_to_remove)

(1990, 1990)

In [ ]:
# 1_00 - 1MB
# 1_000_000 - 10GB

In [ ]:
# 12 13 402 error
# 19 21 SSLCertVerificationError https://stackoverflow.com/questions/10667960/python-requests-throwing-sslerror
# 40 44 ConnectionResetError
#

In [46]:
json_string = json_strings[26616]
image_json = json.loads(json_string)
print(image_json['image'], image_json['url'])

28966 http://www.cinemapassion.com/stickers4/Oggy_et_les_cafards_Le_film-21091115122013.jpg


In [47]:
try:
    r = requests.get(image_json['url'], verify=False, timeout=2)
except requests.exceptions.RequestException as e:  # This is the correct syntax
    print(e)

In [48]:
is_url_image(r)

True

In [49]:
r.status_code

200

In [50]:
# image = Image.open(requests.get(image_json['url'], verify=False, stream=True).raw)
image = Image.open(BytesIO(r.content))
image = image.convert('RGB')
# image = image.resize((image_size, image_size))
image.thumbnail((image_size, image_size))

In [51]:
np.array(image).shape

(509, 512, 3)

In [ ]:
# image

In [ ]:
size = 256
# resample = None
im = image.resize((size, size))

In [ ]:
im.save(os.path.join(image_save_path, image_name))

# Create archive

In [30]:
!tar -czf ycup21-images-1.tgz {image_save_path}

In [31]:
!ls -lh ./ycup21-images-1.tgz

-rw-r--r-- 1 root root 705M Oct  4 17:29 ./ycup21-images-1.tgz


In [33]:
!ls

drive		  images       index_to_remove.npy  ycup21-images-1.tgz
id_to_remove.npy  images.json  sample_data


In [32]:
!cp ./ycup21-images-1.tgz {BASE_PATH_DATA}

In [34]:
!cp ./index_to_remove.npy {BASE_PATH_DATA}

In [35]:
!cp ./id_to_remove.npy {BASE_PATH_DATA}

In [38]:
images = os.listdir(image_save_path)
len(images)

24627

In [39]:
len(images) + len(index_to_remove)

26617